In [ ]:
%load_ext autoreload
%autoreload 2
# use bnn_new_pysr environment for this notebook

In [4]:
import os
os.environ['PYSR_AUTOLOAD_EXTENSIONS'] = 'no'
from matplotlib import pyplot as plt
from brokenaxes import brokenaxes
import pandas as pd
pd.set_option('display.max_colwidth', None)
%matplotlib inline
plt.rcParams["font.family"] = "serif"

In [12]:
from interpret import *

In [6]:
version = 24880
# pysr_version = 11003
pysr_version = 93102

In [7]:
feature_nn = get_feature_nn(version)
# important_complexities = [1, 3, 4, 7, 11, 13, 14, 16, 20, 22, 24, 26]

In [13]:
results = get_pysr_results(pysr_version, include_ssx=False, feature_nn=feature_nn)

TypeError: get_rmse_values() got an unexpected keyword argument 'test'

In [8]:
latex_strs = f2_latex_strings(results)

In [10]:
font_family = 'serif'
format = 'svg'
for c, s in latex_strs.items():
    s = f'${s}$'
    save_latex(s, f'graphics/f2_latex/{version}_{pysr_version}_{c}_{font_family}.{format}', format=format)

In [ ]:
# f2_str = f2_latex_str(results, mapping_dict=None)

NameError: name 'f2_latex_str' is not defined

In [17]:
# f1_str = f1_latex_string2(feature_nn, include_ssx=True, include_ssx_bias=True, pysr_results=results, important_complexities=important_complexities)
# f1_str = f1_latex_string2(feature_nn, include_ssx=True, include_ssx_bias=True, pysr_results=results, important_complexities=important_complexities)
# f1_str_nomap = f1_latex_string(feature_nn, include_ssx=True, include_ssx_bias=True)

In [11]:
f1_strs = f1_latex_strings(feature_nn, include_ssx=True, include_ssx_bias=True, pysr_results=results, important_complexities=None)

In [20]:
save_latex("Learned features:", f'graphics/f1_latex/features.svg')

In [12]:
for i, f1_str in enumerate(f1_strs):
    f1_str = f'${f1_str}$'
    save_latex(f1_str, f'graphics/f1_latex/{i}.svg')

In [9]:
nn_results = pickle.load(open('pickles/nn_results_all_24880.pkl', 'rb'))
petit_results = pickle.load(open('pickles/petit_results_all.pkl', 'rb'))
nn_rmse = nn_results['test']
petit_rmse = petit_results['test']

In [1]:
# period_ratio_rmse = pickle.load(open('pickles/period_ratio_rmse.pkl', 'rb'))

# def plot_period_ratio_rmse():
#     plt.figure(figsize=(6, 4))  # Width = 10 inches, Height = 6 inches
#     ax = plt.gca()
#     eqs = [int(s[2:]) for s in period_ratio_rmse.keys() if s[0:2] == 'eq']
#     rmses = [period_ratio_rmse[f'eq{eq}'] for eq in eqs]
#     ax.plot(eqs, rmses, 'o', label='Equations')
#     ax.set_xlabel('Complexity')
#     ax.set_ylabel('Period ratio sweep RMSE')
#     # add horizontal line at period_ratio_rmse['nn']
#     ax.axhline(period_ratio_rmse['nn'], color='r', linestyle='--', label='NN')
#     ax.legend()
#     # add minor x axis tick marks at integers
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
#     return plt

# def period_bar_chart():
#     nn_rmse = period_ratio_rmse['nn']
#     eq_rmse = period_ratio_rmse['eq26']

#     plt.figure(figsize=(4, 3))  # Width = 10 inches, Height = 6 inches
#     plt.bar(['NN', 'Equations'], [nn_rmse, eq_rmse], color=['blue', 'green'])
#     # add value label above each bar
#     for i, v in enumerate([nn_rmse, eq_rmse]):
#         plt.text(i, v + 0.05, str(round(v, 2)), ha='center', va='bottom')
#     plt.ylabel('RMSE')
#     plt.ylim(0, 2.2)
#     # make y axis ticks just integers
#     plt.gca().yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
#     return plt

# plt = plot_period_ratio_rmse()
# plt.savefig('graphics/period_ratio_rmse.png', dpi=400)
# plot = period_bar_chart()
# plot.savefig('graphics/period_bar_chart.png')
# plot.show()

In [34]:
# making the pareto plot
def make_pareto_plot(results, important_complexities=None, rmse=True, plot_unimportant=False):
    plt.rcParams["font.family"] = "serif"
    plt.figure(figsize=(8, 4.8))  # Width = 10 inches, Height = 6 inches
    ax = plt.gca()
    # ax = brokenaxes(ylims=((1.0, 1.7), (3.2, 3.4)), hspace=0.45)

    if rmse:
        rmse_vals = get_k_rmse_values()[2]
        results['rmse'] = [rmse_vals[c] for c in results['complexity']]

    if plot_unimportant:
        ax.scatter(results['complexity'], results['rmse' if rmse else 'loss'], s=12)

    ax.set_xlabel('Equation complexity', fontsize=12, labelpad=10)

    ax.set_ylabel('RMSE' if rmse else 'Loss', fontsize=12, labelpad=10)

    if important_complexities:
        important_ixs = get_important_ixs(results, important_complexities)
        important_ys = [results['rmse' if rmse else 'loss'][i] for i in important_ixs]
        ax.scatter(important_complexities, important_ys, color='red' if plot_unimportant else None, s=22)
        ax.plot(important_complexities, important_ys, color='red' if plot_unimportant else None, label='Ours')

    # # add horizontal lines for nn and petit values
    # ax.axhline(y=nn_rmse, color='orange', label='NN')
    # ax.axhline(y=petit_rmse, color='purple', label='Petit+ 2020')
    # ax.legend()

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.tick_params(axis='both', which='major', labelsize=10)
    # add minor tick marks
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
    # Make minor ticks point outward
    ax.tick_params(axis="x", which="minor", direction="out")

    plt.ylim(1.25, 1.75)  # Set ymin to 3 and ymax to 7
    return plt


In [10]:
def make_pareto_plot2(results, important_complexities=None, rmse=True, plot_unimportant=False):
    plt.rcParams["font.family"] = "serif"
    plt.figure(figsize=(3, 2.25))
    ax = plt.gca()

    if rmse:
        rmse_vals = get_k_rmse_values()[2]
        results['rmse'] = [rmse_vals[c] for c in results['complexity']]

    if plot_unimportant:
        ax.scatter(results['complexity'], results['rmse' if rmse else 'loss'], s=12)

    ax.set_xlabel('Equation complexity')
    ax.set_ylabel('RMSE' if rmse else 'Loss')

    if important_complexities:
        important_ixs = get_important_ixs(results, important_complexities)
        important_ys = [results['rmse' if rmse else 'loss'][i] for i in important_ixs]
        ax.scatter(important_complexities, important_ys, color='red' if plot_unimportant else None, s=22)
        ax.plot(important_complexities, important_ys, color='red' if plot_unimportant else None,
                label='Distilled Equations')

    # Horizontal lines
    ax.axhline(y=nn_rmse, color='orange', label='Neural Network')
    ax.axhline(y=petit_rmse, color='purple', label='Petit+20')
    ax.axhline(y=2.75, color='green', label='Pure SR')

    # Move legend above lines (ok if it occludes them)
    ax.legend(loc='upper right', bbox_to_anchor=(1, 1.1), framealpha=1)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
    ax.tick_params(axis="x", which="minor", direction="out")

    return plt


In [11]:
plot = make_pareto_plot2(results, important_complexities=None, rmse=True, plot_unimportant=False)
plot.savefig('graphics/small_pareto_plot_new.svg', bbox_inches='tight', format='svg')
plot.show()

KeyError: 21

In [ ]:
plot = make_pareto_plot(results, important_complexities=None, rmse=True, plot_unimportant=False)
plot.savefig('graphics/pareto_fig1_new.svg')
# plot.savefig('graphics/pareto_fig1.png', dpi=400)
# plot.show()


In [20]:
results = pickle.load(open('pickles/nn_and_petit_results_test.pkl', 'rb'))
nn_rmse = results['rmse']
petit_rmse = results['petit_rmse']

def bar_chart():
    rmse_values = get_k_rmse_values()[2]
    eq_rmse = min(rmse_values.values())
    # nn_rmse and petit_rmse already calculated above

    plt.figure(figsize=(4, 3))  # Width = 10 inches, Height = 6 inches
    plt.bar(['NN', 'Equations', 'Petit+ 2020'], [nn_rmse, eq_rmse, petit_rmse], color=['blue', 'green', 'red'])
    # add value label above each bar
    for i, v in enumerate([nn_rmse, eq_rmse, petit_rmse]):
        plt.text(i, v + 0.05, str(round(v, 2)), ha='center', va='bottom')
    plt.ylabel('RMSE')
    plt.ylim(0, 4)
    # make y axis ticks just integers
    plt.gca().yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    plt.savefig('graphics/bar_chart.png')
    plt.show()

bar_chart()